In [58]:
import os
import numpy as np
from datetime import datetime
path = os.getcwd()
if path[1:5] == "Home":
    %run /Home/siv30/wad005/master/GRB/src/harvest_matlab_data.ipynb
    %run /Home/siv30/wad005/master/GRB/src/convert_erg.ipynb
    %run /Home/siv30/wad005/master/GRB/src/safety_time.ipynb
else:
    %run /Users/andreas/phys/master/GRB/src/harvest_matlab_data.ipynb
    %run /Users/andreas/phys/master/GRB/src/convert_erg.ipynb
    %run /Users/andreas/phys/master/GRB/src/safety_time.ipynb

/Home/siv30/wad005/useful_py_functions/matplotlib_custom_style_and_macros.py:128: MatplotlibDeprecationWarning: Support for setting an rcParam that expects a str value to a non-str value is deprecated since 3.5 and support will be removed two minor releases later.
  rcParams['lines.marker'] = None
/Home/siv30/wad005/useful_py_functions/matplotlib_custom_style_and_macros.py:128: MatplotlibDeprecationWarning: Support for setting an rcParam that expects a str value to a non-str value is deprecated since 3.5 and support will be removed two minor releases later.
  rcParams['lines.marker'] = None
/Home/siv30/wad005/useful_py_functions/matplotlib_custom_style_and_macros.py:128: MatplotlibDeprecationWarning: Support for setting an rcParam that expects a str value to a non-str value is deprecated since 3.5 and support will be removed two minor releases later.
  rcParams['lines.marker'] = None



all UiB paths loaded

all UiB paths loaded
read_matlab_file.ipynb loaded successfully!

all UiB paths loaded
read_matlab_file.ipynb loaded successfully!
convert_erg.ipynb loaded successfully!

all UiB paths loaded
read_matlab_file.ipynb loaded successfully!

all UiB paths loaded
read_matlab_file.ipynb loaded successfully!


/Home/siv30/wad005/useful_py_functions/matplotlib_custom_style_and_macros.py:128: MatplotlibDeprecationWarning: Support for setting an rcParam that expects a str value to a non-str value is deprecated since 3.5 and support will be removed two minor releases later.
  rcParams['lines.marker'] = None
/Home/siv30/wad005/useful_py_functions/matplotlib_custom_style_and_macros.py:128: MatplotlibDeprecationWarning: Support for setting an rcParam that expects a str value to a non-str value is deprecated since 3.5 and support will be removed two minor releases later.
  rcParams['lines.marker'] = None


convert_erg.ipynb loaded successfully!
safety_time.ipynb loaded successfully!


In [3]:
#compute len(tus_bkg)/(tus_bkg[-1]-tus_bkg[0]).total_seconds() <--- just the count rate

In [99]:
T0 = datetime.fromisoformat('2019-03-05T13:05:19.816')
print(T0)

2019-03-05 13:05:19.816000


In [100]:
df = harvest_data(fn='2019_3_5_13_3_48',T0='2019-03-05T13:05:19.816')

dau/det starting from index 1
tus array given in seconds
df sorted by tus


In [140]:
i_bkg_end = find_nearest(df['tus'].to_numpy(),-2)
df_bkg = df.truncate(after=i_bkg_end-1)
df_bkg

,year,month,day,hour,minute,second,erg,dau,det,datetime,tus,adr
0,2019,3,5,13,3,47.521628,1397,2,3,2019-03-05 13:03:47.521628443,-92.294372,6
1,2019,3,5,13,3,47.522101,84,2,3,2019-03-05 13:03:47.522100556,-92.293899,6
2,2019,3,5,13,3,47.522307,39,2,2,2019-03-05 13:03:47.522307082,-92.293693,5
3,2019,3,5,13,3,47.522423,28,1,2,2019-03-05 13:03:47.522422528,-92.293577,2
4,2019,3,5,13,3,47.522521,36,2,2,2019-03-05 13:03:47.522520889,-92.293479,5
...,...,...,...,...,...,...,...,...,...,...,...,...
8435,2019,3,5,13,3,49.528722,1528,3,2,2019-03-05 13:03:49.528721694,-90.287278,8
8436,2019,3,5,13,3,49.528722,37,1,2,2019-03-05 13:03:49.528721860,-90.287278,2
8437,2019,3,5,13,3,49.52986,91,2,1,2019-03-05 13:03:49.529859972,-90.286140,4
8438,2019,3,5,13,3,49.53039,35,3,2,2019-03-05 13:03:49.530390389,-90.285610,8


In [141]:
tus,erg,dau,det = df_bkg['tus'].to_numpy(),df_bkg['erg'].to_numpy(),df_bkg['dau'].to_numpy(),df_bkg['det'].to_numpy()

In [142]:
#remove triggers due to safetytime
keV,abc = convert_erg(erg,dau,det,T0)
df_bkg['keV'] = keV

In [143]:
df_bkg = df_bkg.sort_values(by=['adr','tus'],ignore_index=True)
df_bkg

,year,month,day,hour,minute,second,erg,dau,det,datetime,tus,adr,keV
0,2019,3,5,13,3,47.525428,28,1,1,2019-03-05 13:03:47.525427861,-92.290572,1,282.502204
1,2019,3,5,13,3,47.525878,43,1,1,2019-03-05 13:03:47.525877778,-92.290122,1,406.136344
2,2019,3,5,13,3,47.530015,52,1,1,2019-03-05 13:03:47.530014972,-92.285985,1,481.408924
3,2019,3,5,13,3,47.53014,408,1,1,2019-03-05 13:03:47.530140111,-92.285860,1,4115.834284
4,2019,3,5,13,3,47.537834,1768,1,1,2019-03-05 13:03:47.537834472,-92.278166,1,29799.646444
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8435,2019,3,5,13,3,49.49551,36,4,3,2019-03-05 13:03:49.495510250,-90.320490,12,448.472124
8436,2019,3,5,13,3,49.500377,77,4,3,2019-03-05 13:03:49.500376528,-90.315623,12,946.613801
8437,2019,3,5,13,3,49.50333,40,4,3,2019-03-05 13:03:49.503329667,-90.312670,12,496.661500
8438,2019,3,5,13,3,49.503671,59,4,3,2019-03-05 13:03:49.503671361,-90.312329,12,726.771089


In [97]:
def get_flag(df):
    '''improvement needed ...
    df have to be sorted by adr and tus
    
    
    '''
    df = df.sort_values(by=['adr','tus'],ignore_index=True)
    st_arr = df["st"].to_numpy()
    dt_arr = df["dt"].to_numpy()
    adr_arr = df['adr'].to_numpy()
    
    flag = []
    i = 0
    while i < len(st_arr):
        #TODO: handle when adr increments one step
        # loop over the triggers when detector is ready again (check st[i] for how long that is)
        # 
        
        if adr_arr[i] != adr_arr[i-1]:
            flag.append(False)
            i+=1
        elif dt_arr[i] >= st_arr[i-1]:
            flag.append(False)
            i+=1
        elif dt_arr[i] <= st_arr[i-1] and adr_arr[i] == adr_arr[i-1]:
            st = st_arr[i-1]
            diff = dt_arr[i]
            flags = 0
            i += 1
            while diff < st:
                if adr_arr[i] != adr_arr[i-1]:
                    i +=1
                else:
                    diff += dt_arr[i]
                    flags += 1
                    i +=1
            flag.extend(flags*[True])
            i -= 1
        else:
            flag.append(False)
            i += 1
            
    if len(flag) == len(st_arr):
        return flag
    else:
        print("the shape of the flag array is not correct")

In [144]:
df_bkg['tus'] = df_bkg['tus'].to_numpy()*1e6 
st,dt = get_st(df_bkg),get_dt(df_bkg)
df_bkg['st']=st
df_bkg['dt']=dt
#flag = get_flag(df_bkg)
#df_bkg['flag']=flag #assuming get_flag not working correctly

In [145]:
df_bkg.head(10)

,year,month,day,hour,minute,second,erg,dau,det,datetime,tus,adr,keV,st,dt
0,2019,3,5,13,3,47.525428,28,1,1,2019-03-05 13:03:47.525427861,-9.229057e+07,1,282.502204,0.000000,0.000
1,2019,3,5,13,3,47.525878,43,1,1,2019-03-05 13:03:47.525877778,-9.229012e+07,1,406.136344,0.061363,449.917
2,2019,3,5,13,3,47.530015,52,1,1,2019-03-05 13:03:47.530014972,-9.228599e+07,1,481.408924,0.814089,4137.194
3,2019,3,5,13,3,47.53014,408,1,1,2019-03-05 13:03:47.530140111,-9.228586e+07,1,4115.834284,15.000000,125.139
4,2019,3,5,13,3,47.537834,1768,1,1,2019-03-05 13:03:47.537834472,-9.227817e+07,1,29799.646444,25.000000,7694.361
5,2019,3,5,13,3,47.542892,1960,1,1,2019-03-05 13:03:47.542892028,-9.227311e+07,1,34932.203500,26.972881,5057.556
6,2019,3,5,13,3,47.544665,1686,1,1,2019-03-05 13:03:47.544665361,-9.227133e+07,1,27721.215076,25.000000,1773.333
7,2019,3,5,13,3,47.549374,24,1,1,2019-03-05 13:03:47.549374306,-9.226663e+07,1,249.917356,0.000000,4708.945
8,2019,3,5,13,3,47.550274,152,1,1,2019-03-05 13:03:47.550274028,-9.226573e+07,1,1372.881324,4.988700,899.722
9,2019,3,5,13,3,47.551143,24,1,1,2019-03-05 13:03:47.551142778,-9.226486e+07,1,249.917356,0.000000,868.750


In [146]:
dt,st,adr = df_bkg['dt'].to_numpy(dtype=np.float64),df_bkg['st'].to_numpy(dtype=np.float64),df_bkg['adr'].to_numpy()

In [131]:
adr_1,st_1,dt_1 = adr[np.where(adr==1)[0]],st[np.where(adr==1)[0]],dt[np.where(adr==1)[0]]

In [200]:
def get_flag(df):
    '''
    df['tus'] must be given in microseconds
    '''
    df = df.sort_values(by=['adr','tus'])
    dt,st,adr = df['dt'].to_numpy(dtype=np.float64),df['st'].to_numpy(dtype=np.float64),df['adr'].to_numpy()
    
    flag = []
    #building this shit for the WHOLE array
    i=0
    while i<len(dt):
         #handling start
        if i == 0:
            print('start')
            flag.append(False)
            i+=1

        #handling detector change        
        elif adr[i] != adr[i-1]:
            print('adr')
            flag.append(False)
            i+=1
            
        #handling regular st,dt comparison    
        elif st[i-1] >= dt[i]:
            print('handling')
            st_start = st[i-1]
            diff = dt[i]
            flags = 0
            i += 1
            while diff < st_start: #handling severe dead time due to large safetytime
                if adr[i] != adr[i-1]:
                    break
                else:
                    diff += dt[i]
                    flags += 1
                    i +=1
            flag.extend(flags*[True])
            i -= 1 #problem now with the for loop is that is begins for the start again, even though it have looped over n counts
        
        #handling end of array        
        elif i == len(dt)-1:
            print('end',i)
            if st[i-1] >= dt[i]:
                flag.append(True)
                if len(flag) != len(dt):
                    print('length of flag and dt are different')
                return np.array(flag)
            else:
                flag.append(False)
                if len(flag) != len(dt):
                    print('length of flag and dt are different')
                return np.array(flag)
        else:
            flag.append(False)
            i+=1

In [210]:
print(len(np.where(flag==True)[0]))
np.where(flag==True)[0]

13


array([ 338,  611, 2267, 2624, 3445, 3742, 3858, 4271, 4321, 5226, 5456,
       8060, 8399])

In [198]:
len(flag)-len(df_bkg['dt'].to_numpy())

0

In [202]:
df_bkg['flag'] = flag

In [214]:
print('% removed: ',(13/len(dt))*100)

% removed:  0.15402843601895735


In [215]:
print(df_bkg.shape)
df_bkg.drop(labels=np.where(flag==True)[0],inplace=True)
print(df_bkg.shape)

(8440, 16)
(8427, 16)


In [219]:
df_bkg = df_bkg.sort_values(by='tus')
tus = df_bkg['tus'].to_numpy(dtype=np.float64)
exposure = (tus[-1]-tus[0])/1e6 #seconds
print(np.round(len(tus)/exposure,3))

4194.389


In [220]:
#TODO: wrap this whole thing into functions for quick access to background rates